In [1]:
import numpy as np
from joblib import dump
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge, RidgeClassifier
from sklearn.metrics import mean_squared_error, classification_report

In [2]:
with open("processed_data/grade.npy", "rb") as f:
    score = np.load(f)

with open("processed_data/score.npy", "rb") as f:
    grade = np.load(f)

In [3]:
with open("processed_data/ingredients.npy", "rb") as f:
    ingredients = np.load(f, allow_pickle=True)

with open("processed_data/products.npy", "rb") as f:
    products = np.load(f, allow_pickle=True)

In [4]:
ingredients_selection = Counter(ingredients).most_common()[:10000]
top_ingredients = [x[0] for x in ingredients_selection]

In [5]:
%%time
products_map = np.array([1 * np.isin(top_ingredients, product) for product in products])

Wall time: 6min 37s


In [6]:
X_train, X_test, y_train, y_test = train_test_split(products_map, grade, test_size=.3, random_state=101, stratify=grade)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=.5, random_state=101, stratify=y_test)

In [7]:
reg = Ridge(alpha=5)
reg.fit(X_train, y_train)
reg.score(X_val, y_val)

0.5805426555407531

In [8]:
pred = reg.predict(X_test)

In [9]:
mean_squared_error(y_test, pred)

0.7910499648316448

In [129]:
# a = reg.coef_
# b = reg.coef_
c = reg.coef_

In [134]:
from scipy.stats import kendalltau
kendalltau(a, c[:500])

KendalltauResult(correlation=0.7817074148296592, pvalue=1.7624781295311126e-150)

In [122]:
len(b)

1000

In [7]:
rc = RidgeClassifier(alpha=0.1)
rc.fit(X_train, y_train)
rc.score(X_val, y_val)

MemoryError: Unable to allocate 13.0 GiB for an array with shape (174486, 10000) and data type float64

In [ ]:
pred = rc.predict(X_test)

In [ ]:
print(classification_report(y_test, pred))

In [62]:
%%time
A = np.array(X_train)
At = A.transpose()

Wall time: 224 ms


In [63]:
%%time
x = At.dot(A)

Wall time: 7min 8s


In [64]:
%%time
x = np.linalg.inv(x)

Wall time: 42.1 ms


In [65]:
%%time
x = x.dot(At)

Wall time: 552 ms


In [66]:
%%time
x = x.dot(y_train)

Wall time: 53.5 ms


In [67]:
pred = np.array(X_test).dot(x)

In [68]:
mean_squared_error(y_test, pred)

1.840254820439007